# Random Forest

#### Importing Libraries & Data

In [1]:
import pandas as pd
import json
import numpy as np
import pickle

import time
np.random.seed(42)
from numpy.core.umath_tests import inner1d

from sklearn.feature_extraction import stop_words
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier

import regex as re
import matplotlib.pyplot as plt

/Users/benjaminnew/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  


In [2]:
X = pd.read_csv('../Data/X.csv')
y = pd.read_csv('../Data/y.csv', header = None)

## Merging  my text and Train test splitting

In [3]:
X_corpus  = X['selftext'] + ' ' + X['title']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_corpus,y)

In [5]:
my_stopwords = list(stop_words.ENGLISH_STOP_WORDS) + ['crypto' , 'cryptocurrency' , 'cryptocurrencies', 'blockchain','bitcoin'
              ,'stock' , 'stocks'
             , 'https', 'io','fintel','amp','com','13d','8209','08','gt','www','000','http','07',
             'x200b','daex', 'click']

## Setting hyperparameters for GridSearch

### Random forests, are mostly used  as classifiers and regressors
####  1) 'n_estimators' =  The number of Decisions Trees that my Grid Search will produce
####  2) ' min_samples_split' = The Minimum number of samples required to split an internal node
####  3) ' max_featuers' = The maximum number of featuers that the TFIFD should use

In [6]:
params = {'RF__n_estimators' : [100,130,150],
          'RF__min_samples_split': [2 , 3 , 4],
          'TFIDF__max_features': [500,700,1000]}

### Setting pipeline with TFIDF and Random Forest

In [7]:
pipe = Pipeline([
    ('TFIDF', TfidfVectorizer(stop_words=my_stopwords)),
    ('RF', RandomForestClassifier())
])

#### GridSearch with my Pipe and parameters

In [25]:
gs = GridSearchCV(pipe, params)

In [26]:
y_train_good = y_train.values.ravel()

In [27]:
gs.fit(X_train, y_train_good)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('TFIDF', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'RF__n_estimators': [100, 130, 150], 'RF__min_samples_split': [2, 3, 4], 'TFIDF__max_features': [500, 700, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

#### Best parameters to use for model

In [28]:
gs.best_params_

{'RF__min_samples_split': 3,
 'RF__n_estimators': 150,
 'TFIDF__max_features': 1000}

In [29]:
gs.score(X_train, y_train)

0.9968553459119497

In [30]:
gs.score(X_test, y_test)

0.8606403013182674

In [31]:
with open('../Data/gs.pkl', 'wb+') as f:
    pickle.dump(gs, f)

## Feature importance shows me the most important features(words) that help the decision tree make decisions on where to split and where to put certain posts

In [32]:
feat_importance = pd.DataFrame(gs.best_estimator_.named_steps['RF'].feature_importances_, 
             gs.best_estimator_.named_steps['TFIDF'].get_feature_names(), 
             columns=['importance'])

#### Viewing my Feature importance

In [33]:
(feat_importance['importance']==0).sum()

44

In [34]:
feat_importance.shape

(1000, 1)

In [35]:
feat_importance.sort_values('importance', ascending=False).head(10)

,importance
company,0.030219
earnings,0.027027
market,0.024759
thoughts,0.022923
invest,0.019490
investing,0.019402
buy,0.019361
trade,0.014680
think,0.013388
today,0.013361


## Creating Dataframe with the probability that a post in a certain is classified in a certain class, along with what true class it belonged to

In [36]:
proba_df = pd.DataFrame(X_test)
proba_df.reset_index(drop=True, inplace=True)

In [37]:
proba_df['proba'] = gs.predict_proba(X_test)[:, 1]

In [38]:
proba_df['true_y'] = y_test.reset_index(drop=True)

## False Negatives

In [39]:
proba_df[(proba_df['true_y']==1) & (proba_df['proba']<.5)].sort_values('proba').sum()

0           Is tony dwyer a trusted analyst, has he been...
proba                                               12.3439
true_y                                                   36
dtype: object

In [29]:
False_Negatives = proba_df[(proba_df['true_y']==1) & (proba_df['proba']<.5)].sort_values('proba')

## False Positives

In [34]:
proba_df[(proba_df['true_y']==0) & (proba_df['proba']>.5)].sort_values('proba').head()

,0,proba,true_y
136,This is getting out of hand...we have got to...,0.500282,0
202,Will you buy your dream car with cryptocurre...,0.506199,0
62,"Wirex Ethereum wallet allows you to buy, sel...",0.512556,0
453,"We created an ""alt-coin correction index"". R...",0.513483,0
284,ARYZE - Winner of the 2018 Best New Danish S...,0.519111,0


## Area of low confidence

In [35]:
proba_df[(proba_df['proba']>.45) & (proba_df['proba']<.55)].sort_values('proba').head()

,0,proba,true_y
397,If you bought at the top of the crypto bubbl...,0.458829,0
18,A visualization of the cryptocurrency market...,0.466224,0
342,I am investor in Turkey and wanted to see how ...,0.469793,1
454,[Behold the scariest chart for the stock marke...,0.471620,1
122,"Kinesis, how to buy the KVTs step-by-step",0.475367,0
